# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key



### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_data_path = 'output_data/cities.csv'

# Read the mouse data and the study results
cities_data = pd.read_csv(cities_data_path)

cities_data.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Port Hardy,50.6996,-127.4199,285.15,71,40,3.60,CA,Tue May 4 14:38:47 2021
1,1,Port Augusta,-32.5000,137.7667,285.15,66,3,5.14,AU,Tue May 4 14:38:48 2021
2,2,Hermanus,-34.4187,19.2345,292.04,82,18,1.79,ZA,Tue May 4 14:38:48 2021
3,3,Malé,4.1748,73.5089,302.15,84,75,1.03,MV,Tue May 4 14:38:49 2021
4,4,Registro,-24.4875,-47.8436,301.63,52,8,0.93,BR,Tue May 4 14:38:49 2021


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(g_key)

locations = cities_data[["Lat", "Lng"]].astype(float)

humidity_rate = cities_data["Humidity"].astype(float)

In [4]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity_rate, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [11]:


new_cities_df = cities_data.loc[(cities_data["Max Temp"] > 70) & 
                           (cities_data["Wind Speed"] < 10) &
                                    (cities_data["Cloudiness"] == 0), :]
new_cities_df.dropna(how ='any')



,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
23,23,Saldanha,-33.0117,17.9442,288.15,82,0,1.54,ZA,Tue May 4 14:38:55 2021
24,24,Guane,22.2025,-84.0875,305.28,50,0,7.47,CU,Tue May 4 14:34:27 2021
27,27,Bełchatów,51.3688,19.3567,287.15,41,0,4.12,PL,Tue May 4 14:38:57 2021
31,31,Grand Gaube,-20.0064,57.6608,298.71,88,0,0.45,MU,Tue May 4 14:38:58 2021
40,40,East London,-33.0153,27.9116,292.15,63,0,2.57,ZA,Tue May 4 14:39:02 2021
...,...,...,...,...,...,...,...,...,...,...
527,527,Grushevskaya,47.4406,39.9511,284.82,43,0,3.00,RU,Tue May 4 14:41:50 2021
528,528,Binzhou,37.3667,118.0167,283.34,35,0,4.12,CN,Tue May 4 14:41:51 2021
531,531,Marsá Maţrūḩ,31.3525,27.2453,297.15,60,0,5.14,EG,Tue May 4 14:37:11 2021
535,535,Winneba,5.3511,-0.6231,301.15,78,0,7.20,GH,Tue May 4 14:41:53 2021


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [12]:
hotel_df = new_cities_df[["City", "Country", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Lat,Lng,Hotel Name
23,Saldanha,ZA,-33.0117,17.9442,
24,Guane,CU,22.2025,-84.0875,
27,Bełchatów,PL,51.3688,19.3567,
31,Grand Gaube,MU,-20.0064,57.6608,
40,East London,ZA,-33.0153,27.9116,
...,...,...,...,...,...
527,Grushevskaya,RU,47.4406,39.9511,
528,Binzhou,CN,37.3667,118.0167,
531,Marsá Maţrūḩ,EG,31.3525,27.2453,
535,Winneba,GH,5.3511,-0.6231,


In [13]:
params = {
    "radius": 50000,
    "types": "hotel",
    "key": g_key
}

for index, row in hotel_df.iterrows():
    
    lat = row["Lat"]
    lng = row["Lng"]

    params["location"] = f"{lat},{lng}"

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    name_address = requests.get(base_url, params=params)
    
    
    name_address = name_address.json()


    
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
        
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

In [14]:

info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""

hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [15]:
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)


fig.add_layer(heat_layer)
fig.add_layer(markers)


fig

Figure(layout=FigureLayout(height='420px'))